In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install tensorflow_datasets

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
import tensorflow as tf


In [ ]:
import tensorflow_datasets as tfds
from keras.utils.vis_utils import plot_model

In [ ]:
SIZE = 64

train_images = []
train_labels = [] 
for directory_path in glob.glob(r"/content/drive/MyDrive/dataset/dataset_split/train/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
#print(train_images)
train_images = np.array(train_images)
print(train_images[0])
train_labels = np.array(train_labels)

In [ ]:
print(train_labels[0])
print(len(set(train_labels)))

In [ ]:
# test
test_images = []
test_labels = [] 
for directory_path in glob.glob(r"/content/drive/MyDrive/dataset/dataset_split/test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
print(len(set(test_labels)))

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
print(train_labels_encoded)

In [ ]:
#Split data
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:
#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

Feature Extractor


In [ ]:

VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()
feature_extractor=VGG_model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
print(len(VGG_model.layers))
X_test=VGG_model.predict(x_test)
X_for_RF = features

In [ ]:
VGG_model.save('vgg16_feature_extractor.h5')

In [ ]:
import keras
def load_model(path):
    model = keras.models.load_model(path)
    #model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    return model
path=r"/content/vgg16_feature_extractor.h5"
model2=load_model(path)

In [ ]:
model2.predict(x_train)

In [ ]:
X_test=X_test.reshape(X_test.shape[0], -1)

In [ ]:
print(X_for_RF.shape)
print(X_test.shape)

# stacking

In [ ]:
import sklearn.metrics as metrics
pred_rf=rf.predict(val_features)
print(metrics.accuracy_score(pred_rf,val_labels))

In [ ]:
import joblib
joblib.dump(rf, 'rf_model.pkl')


In [ ]:
svm = SVC(kernel='linear', probability=True, random_state=42)
svm.fit(train_features, train_labels)

In [ ]:
pred_svm=svm.predict(val_features)
print(metrics.accuracy_score(pred_svm,val_labels))

In [ ]:
import joblib
joblib.dump(svm,'svm_model.pkl')

In [ ]:
xgb = XGBClassifier(random_state=42)
xgb.fit(train_features, train_labels)

In [ ]:
pred_xgb = xgb.predict(val_features)
print(metrics.accuracy_score(pred_xgb,val_labels))

In [ ]:
import joblib
joblib.dump(xgb,'xgb_model.pkl')

In [ ]:
train_predrf=rf.predict(train_features)
train_predsvm=svm.predict(train_features)
train_predxgb=xgb.predict(train_features)

In [ ]:
# Concatenate the base classifier predictions with the VGG16 features
#base_preds = np.column_stack((pred_rf, pred_svm,pred_xgb))
b_preds=np.column_stack((train_predrf,train_predsvm,train_predxgb))
print(b_preds.shape)
input_data = np.concatenate((train_features, b_preds), axis=1)
print(input_data.shape)

In [ ]:
print(train_features.shape)
print(val_features.shape)
print(train_labels.shape)
print(val_labels.shape)

In [ ]:
# Define the model
timesteps=1
input_dim=4096
num_classes=10
batch_size=32
epochs=1

model = Sequential()
model.add(LSTM(128, input_shape=(1, input_data.shape[1])))
model.add(Dense(10, activation='softmax'))

In [ ]:
from keras.optimizers import Adam
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(np.expand_dims(input_data, axis=1), y_train_one_hot, batch_size=32, epochs=5)

In [ ]:
X_test_stacked = np.column_stack((pred_rf, pred_svm, pred_xgb))
xtest=np.concatenate((val_features, X_test_stacked), axis=1)

In [ ]:
# Use the LSTM classifier to make predictions on the validation set
lstm_preds = np.round(model.predict(xtest[:,np.newaxis,:]))

In [ ]:
model.save("stackedLSTM.h5")

In [ ]:
# Evaluate the LSTM predictions
accu = np.mean(ymodel1_preds == y_test_one_hot)
print(f'LSTM accuracy: {accu}')

Soft Voting

In [ ]:
import keras
clf4 = keras.models.load_model(r'/content/drive/MyDrive/dataset/LSTMmodel.h5')
# Predict the class probabilities for each classifier
y_proba4 = clf4.predict(np.expand_dims(val_features, axis=1))




In [ ]:
# Combine the predictions of the base classifiers using majority voting
majority_preds = np.array([np.argmax(np.bincount(preds.astype(int))) for preds in np.column_stack((pred_rf,pred_svm,pred_xgb))],dtype=int)

# Evaluate the majority voting ensemble on the validation set
accuracy = np.mean(majority_preds == val_labels)
print('Majority Voting Accuracy:', accuracy)

UI

In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
import numpy as np
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model

In [2]:
from tensorflow.keras.optimizers import Adam
from keras.utils import load_img, img_to_array 
import matplotlib.pyplot as plt
import numpy as np
from tkinter import *
from tkinter.filedialog import askopenfile

In [3]:
import PIL.Image

In [4]:
import keras
def load_model(path):
    model = keras.models.load_model(path)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    return model
path=r"D:\Dataset\Models\stackedLSTM.h5"
model1=load_model(path)

In [5]:
def load_model(path):
    model = keras.models.load_model(path)
    #model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    return model
path=r"D:\Dataset\Models\vgg16_feature_extractor.h5"
model2=load_model(path)

In [6]:
labels_dict= {0:'AnnualCrop', 1:'Forest', 2:'HerbaceousVegetation', 3:'Highway',
       4:'Industrial', 5:'Pasture', 6:'PermanentCrop', 7:'Residential', 8:'River',
       9:'SeaLake'}

In [7]:
from joblib import load
loaded_model1 = load(r'D:\Dataset\Models\rf_model.pkl')
loaded_model2=load(r'D:\Dataset\Models\svm_model.pkl')
loaded_model3=load(r'D:\Dataset\Models\xgb_model.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results

[16:39:14] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [63]:
def predict(imagePath):
    test_image = load_img(imagePath, target_size = (64,64)) 
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    img_feature_extractor=model2.predict(test_image)
    img_feature=img_feature_extractor.reshape(img_feature_extractor.shape[0], -1)
    feat1=loaded_model1.predict(img_feature)
    feat2=loaded_model2.predict(img_feature)
    feat3=loaded_model3.predict(img_feature)
    test_image_stacked = np.column_stack((feat1,feat2,feat3))
    test_image_feat=np.concatenate((img_feature,test_image_stacked), axis=1)
    print(test_image_feat.shape)
    result = model1.predict(test_image_feat[:,np.newaxis,:])
    monocot=labels_dict[np.argmax(result)]
    print(monocot)
    l2 = tk.Label(my_w,text="This is "+monocot,font=my_font1,bg="#ADEFD1", fg="#00203F")
    imagepath=""
    l2.place(x=325,y=275)

In [64]:
def upload_file():
    f_types = [('Jpg Files', '*.jpg')]   # type of files to select
    global filename
    filename = tk.filedialog.askopenfilename(multiple=True,filetypes=f_types)

    for f in filename:
        img=PIL.Image.open(f) # read the image file
        img=img.resize((64,64)) # new width & height
        img=ImageTk.PhotoImage(img)
        e1 =tk.Label(my_w)
        e1.place(x=365,y=140)

        e1.image = img
        e1['image']=img
 # Keep the window open

In [65]:
my_w = tk.Tk()
my_w.geometry("800x350")  # Size of the window
my_w.configure(bg='#ADEFD1')
my_w.title('Land classification')
my_font1=('times', 14, 'bold')
my_font2=('times',18,'bold')
l1 = tk.Label(my_w,text='Upload Files & display',font=my_font1,bg="#ADEFD1", fg="#00203F")
l2 = tk.Label(my_w,text='A Hybrid model built on VGG16 and LSTM for Land classification',font=my_font2,bg="#ADEFD1", fg="#00203F")
l2.place(x=50,y=10)
l1.place(x=300,y=70)
b1 = tk.Button(my_w, text='Choose File',
   width=20,command = lambda:upload_file(),bg="#00203F", fg="#ADEFD1")
b1.place(x=325,y=100)
b2 = tk.Button(my_w, text='Predict',
       width=20,command = lambda:predict(filename[0]),bg="#00203F", fg="#ADEFD1")
b2.place(x=325,y=230)
my_w.mainloop() 

1/1 [==============================] - 0s 78ms/step
(1, 2051)
1/1 [==============================] - 0s 56ms/step
Highway
